In [1]:
import pandas as pd

In [2]:
file_path = ".\Data From Client\SchoolData\Student Survey - Jan.xlsx"

friendships = pd.read_excel(file_path, sheet_name="net_0_Friends")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
friendships.replace(IDtoName, inplace=True)
friendships.head()

,Source,Target
0,Phillip Lozano,Anders Crosby
1,Connor Young,Isaac Nelson
2,Connor Young,Julian King
3,Connor Young,Jeremiah Myers
4,Connor Young,Ignatius Gaines


In [3]:
## Creating graph from the data

import networkx as nx

G = nx.Graph()

for i in range(len(friendships)):
    G.add_edge(friendships['Source'][i], friendships['Target'][i])

In [4]:
from pyvis.network import Network
nt = Network()
nt.from_nx(G)

In [5]:
# # Set the layout algorithm to default which is ForceAtlas2
# nt.save_graph("default.html")

# # Set the layout algorithm to Fruchterman-Reingold
# nt.barnes_hut(gravity=-8000, central_gravity=0.3, spring_length=100, spring_strength=0.001, damping=0.09)
# nt.save_graph("FR_layut.html")

# # Set the layout algorithm to Kamada-Kawai
# pos = nx.kamada_kawai_layout(G)

# # Set node positions manually in Pyvis
# for node, coords in pos.items():
#     for node_dict in nt.nodes:
#         if node_dict['id'] == node:
#             node_dict['x'] = coords[0]
#             node_dict['y'] = coords[1]
#             break
# # Save the graph with Kamada-Kawai-like layout
# nt.save_graph("KK_layout.html")


## We'll use spectral layout as it is a good layout for visualizing social networks and other graphs with a natural community structure.
## It handels large graphs well and is a good choice for visualizing graphs with a natural community structure.

## Circular Layout
# pos = nx.circular_layout(G)
# net = Network()
# net.from_nx(G)
# net.save_graph("circulant_layout.html")

# Set the layout algorithm to Spectral
pos = nx.spectral_layout(G)
# Set node positions manually in Pyvis
for node, coords in pos.items():
    for node_dict in nt.nodes:
        if node_dict['id'] == node:
            node_dict['x'] = coords[0]
            node_dict['y'] = coords[1]
            break
# Save the graph with Spectral-like layout
nt.save_graph("./Layouts/Spectral_layout.html")

In [6]:
G.nodes()

NodeView(('Phillip Lozano', 'Anders Crosby', 'Connor Young', 'Isaac Nelson', 'Julian King', 'Jeremiah Myers', 'Ignatius Gaines', 'Sebastian Gonzalez', 'Kieran Pruitt', 'Thomas Carter', 'Declan Patterson', 'Derek Barnes', 'Bryce Montgomery', 'Chance Wallace', 'Michael Jackson', 'Jude Graham', 'Henry Wright', 'Zayden Coleman', 'Kingston Hamilton', 'Brody Flores', 'Flynn Stuart', 'Emmanuel Ford', 'Andrew Martinez', 'Dominic Rogers', 'Elias Cooper', 'Spencer Woodward', 'Christian Campbell', 'Jason Bailey', 'Patrick Olsen', 'Josiah Perez', 'Rory Briggs', 'Lincoln Peterson', 'Blake Sanders', 'Leo Brewer', 'Malik Cortez', 'Ethan Williams', 'Killian Kent', 'Axel Griffin', 'Jordan Reed', 'Mason Wilson', 'Tate Peck', 'Christopher Clark', 'Cole Brooks', 'Atticus Osborne', 'Graham Glenn', 'Hayden West', 'Walter Kim', 'Aiden Brown', 'Joseph Thompson', 'Finn Perry', 'Cooper Kelly', 'James Anderson', 'Samuel Harris', 'Abel Mendoza', 'Jett Nguyen', 'Miles Watson', 'Chase Wood', 'Adam Rivera', 'Cameron

In [7]:
number_of_nodes = G.number_of_nodes()
number_of_edges = G.number_of_edges()
average_degree = sum(dict(G.degree()).values())/G.number_of_nodes()
density = nx.density(G)
clustering_coefficient = nx.average_clustering(G)
average_shortest_path_length = nx.average_shortest_path_length(G)
diameter = nx.diameter(G)
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
pagerank = nx.pagerank(G)
hits = nx.hits(G)

statistics = {
    'Number of nodes': number_of_nodes,
    'Number of edges': number_of_edges,
    'Average degree': average_degree,
    'Density': density,
    'Clustering coefficient': clustering_coefficient,
    'Average shortest path length': average_shortest_path_length,
    'Diameter': diameter,
    'Degree centrality': degree_centrality,
    'Closeness centrality': closeness_centrality,
    'Betweenness centrality': betweenness_centrality,
    'Eigenvector centrality': eigenvector_centrality,
    'PageRank': pagerank,
    'HITS': hits
}

vals_to_show = ['Number of nodes', 'Number of edges', 'Average degree', 'Density', 'Clustering coefficient', 'Average shortest path length', 'Diameter']

## Adding statistics to the graph
with open("./Layouts/Spectral_layout.html", "r") as file:
    data = file.readlines()

data.insert(1, "<h2>Statistics</h2>\n")
data.insert(2, "<ul>\n")
for key in vals_to_show:
    value = statistics[key]
    data.insert(3, f"<li>{key}: {value:.4f}</li>\n")
        
data.insert(2 + len(statistics) + 1, "</ul>\n")

with open("./Layouts/Spectral_layout.html", "w") as file:
    file.writelines(data)

In [8]:
## Coloring based on degree centrality
max_degree = max(degree_centrality.values())
min_degree = min(degree_centrality.values())
degree_centrality_colors = {node: (degree_centrality[node] - min_degree)/(max_degree - min_degree) for node in degree_centrality}

for node_dict in nt.nodes:
    node = node_dict['id']
    node_dict['color'] = f"rgba(0, 0, 255, {degree_centrality_colors[node]})"
    node_dict['size'] = 10 + 100*degree_centrality[node]

nt.save_graph("./Layouts/Spectral_layout_2.html")

## Community Analysis

In [9]:
## Community analysis
from community import community_louvain
partition = community_louvain.best_partition(G)

for node_dict in nt.nodes:
    node = node_dict['id']
    node_dict['color'] = f"rgba({partition[node]*255}, 0, 0, 1)"
    node_dict['size'] = 10 + 100*degree_centrality[node]

nt.save_graph("./Layouts/Spectral_Community.html")

In [10]:
# Community detection
from networkx.algorithms import community

communities = community.louvain_communities(G)
number_of_communities = len(communities)
community_sizes = [len(community) for community in communities]
largest_community = max(community_sizes)
smallest_community = min(community_sizes)
modularity = community.modularity(G, communities)

## More comminuti Statistics


community_statistics = {
    'Number of communities': number_of_communities,
    'Community with the largest size': largest_community,
    'Community with the smallest size': smallest_community,
    'Modularity': modularity
}

# print(communities)
print("Number of communities: ", number_of_communities)
print("Community with the largest size: ", largest_community)
print("Community with the smallest size: ", smallest_community)
print("Modularity: ", modularity)

Number of communities:  8
Community with the largest size:  33
Community with the smallest size:  11
Modularity:  0.5366111678449117


In [11]:
import matplotlib.colors as mcolors
import numpy as np


# Generate a LinearSegmentedColormap with pastel colors
pastel_cmap = mcolors.LinearSegmentedColormap.from_list(
    'pastel_cmap', 
    np.random.rand(number_of_communities, 4), 
    N=number_of_communities
)

# Get the pastel colors
pastel_colors = pastel_cmap(np.linspace(0, 1, number_of_communities))

In [12]:
# Set node colors based on communities

node_comunity_color_map = dict()

for idx, community in enumerate(communities):
    color = mcolors.to_hex(pastel_colors[idx])
    for node in community:
        node_comunity_color_map[node] = color
for idx, node_dict in enumerate(nt.nodes):
    nt.nodes[idx]['color'] = node_comunity_color_map[node_dict['id']]
# Visualize the network
nt.save_graph("./Layouts/communities.html")



## Adding community statistics to the Graph
with open("./Layouts/communities.html", "r") as file:
    data = file.readlines()

data.insert(1, "<h2>Community Statistics</h2>\n")
data.insert(2, "<ul>\n")
for key, value in community_statistics.items():
    data.insert(3, f"<li>{key}: {value:.4f}</li>\n")
data.insert(2 + len(community_statistics) + 1, "</ul>\n")

with open("./Layouts/communities.html", "w") as file:
    file.writelines(data)

# Answering Questions

In [13]:
import json
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [14]:
## Get all sheet names from excel file
file_path = ".\Data From Client\SchoolData\Student Survey - Jan.xlsx"
df = pd.ExcelFile(file_path)
sheet_names = df.sheet_names

print(sheet_names)

['data_dictionary', 'affiliations', 'participants', 'responses', 'net_0_Friends', 'net_1_Influential', 'net_2_Feedback', 'net_3_MoreTime', 'net_4_Advice', 'net_5_Disrespect', 'net_affiliation_0_SchoolActivit']


## Who are your closest friends?

In [15]:
friendships = pd.read_excel(file_path, sheet_name="net_0_Friends")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
friendships.replace(IDtoName, inplace=True)

G = nx.Graph()

for i in range(len(friendships)):
    G.add_edge(friendships['Source'][i], friendships['Target'][i])


## Based on the graph, let us answer Who are the closest friends for each student?
# For each student -> Save closest friends in a list and then save as a jjson file
closest_friends_data = dict()
for student in G.nodes:
    # Get the shortest paths from this student to all other students
    paths = nx.single_source_shortest_path(G, student)
    
    # The closest friends are those at a distance of 2 (i.e., one intermediate node)
    closest_friends = sorted([(friend, len(path)) for friend, path in paths.items()], key=lambda x: x[1])[:3]
    closest_friends = [friend for friend, distance in closest_friends]
    closest_friends_data[student] = closest_friends

print(closest_friends_data)

with open("./Answering Questions/closest_friends.json", "w") as file:
    json.dump(closest_friends_data, file)


## Save graph using spectral layout
pos = nx.spectral_layout(G)
nt = Network()
nt.from_nx(G)

# Set node positions manually in Pyvis
for node, coords in pos.items():
    for node_dict in nt.nodes:
        if node_dict['id'] == node:
            node_dict['x'] = coords[0]
            node_dict['y'] = coords[1]
            break
# Save the graph with Spectral-like layout
nt.save_graph("./Layouts/Friendships.html")

{'Phillip Lozano': ['Phillip Lozano', 'Anders Crosby', 'Austin Foster'], 'Anders Crosby': ['Anders Crosby', 'Phillip Lozano', 'Austin Foster'], 'Connor Young': ['Connor Young', 'Isaac Nelson', 'Julian King'], 'Isaac Nelson': ['Isaac Nelson', 'Connor Young', 'Kingston Hamilton'], 'Julian King': ['Julian King', 'Connor Young', 'Kieran Pruitt'], 'Jeremiah Myers': ['Jeremiah Myers', 'Connor Young', 'Ignatius Gaines'], 'Ignatius Gaines': ['Ignatius Gaines', 'Connor Young', 'Matthew Martin'], 'Sebastian Gonzalez': ['Sebastian Gonzalez', 'Connor Young', 'Anthony Hill'], 'Kieran Pruitt': ['Kieran Pruitt', 'Connor Young', 'Jeremiah Myers'], 'Thomas Carter': ['Thomas Carter', 'Connor Young', 'Austin Foster'], 'Declan Patterson': ['Declan Patterson', 'Connor Young', 'Lincoln Peterson'], 'Derek Barnes': ['Derek Barnes', 'Bryce Montgomery', 'Chance Wallace'], 'Bryce Montgomery': ['Bryce Montgomery', 'Derek Barnes', 'Maxwell Richardson'], 'Chance Wallace': ['Chance Wallace', 'Derek Barnes', 'Declan 

## Which other students do you to go for advice about schoolwork?

In [16]:
advices = pd.read_excel(file_path, sheet_name="net_4_Advice")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
advices.replace(IDtoName, inplace=True)

G = nx.Graph()

for i in range(len(advices)):
    G.add_edge(advices['Source'][i], advices['Target'][i])

advices_data = dict()

for student in G.nodes:
    # Get the shortest paths from this student to all other students
    paths = nx.single_source_shortest_path(G, student)
    
    # The closest friends are those at a distance of 2 (i.e., one intermediate node)
    closest_advisors = sorted([(friend, len(path)) for friend, path in paths.items()], key=lambda x: x[1])[:3]
    closest_advisors = [advisor for advisor, distance in closest_advisors]
    advices_data[student] = closest_friends

print(advices_data)

with open("./Answering Questions/advices.json", "w") as file:
    json.dump(advices_data, file)

## Save graph using spectral layout
pos = nx.spectral_layout(G)
nt = Network()
nt.from_nx(G)

# Set node positions manually in Pyvis
for node, coords in pos.items():
    for node_dict in nt.nodes:
        if node_dict['id'] == node:
            node_dict['x'] = coords[0]
            node_dict['y'] = coords[1]
            break
# Save the graph with Spectral-like layout
nt.save_graph("./Layouts/Advices.html")

{'Connor Young': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Sebastian Gonzalez': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Kieran Pruitt': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Derek Barnes': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Bennett Sullivan': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Zayden Coleman': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Kingston Hamilton': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Spencer Woodward': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Christian Campbell': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Elias Cooper': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Andrew Martinez': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Dominic Rogers': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Axel Griffin': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Tate Peck': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Graham Glenn': ['Jorge Fox', 'Noah Johnson', 'Tucker Castro'], 'Christop

## Which students are disrespectful towards you?

In [17]:
disrespects = pd.read_excel(file_path, sheet_name="net_5_Disrespect")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
disrespects.replace(IDtoName, inplace=True)

G = nx.Graph()

for i in range(len(disrespects)):
    G.add_edge(disrespects['Source'][i], disrespects['Target'][i])

disrespects_data = dict()

for student in G.nodes:
    # Get the shortest paths from this student to all other students
    paths = nx.single_source_shortest_path(G, student)
    
    # The closest friends are those at a distance of 2 (i.e., one intermediate node)
    disrespectful_students = sorted([(friend, len(path)) for friend, path in paths.items()], key=lambda x: x[1])[:3]
    disrespectful_students = [advisor for advisor, distance in disrespectful_students]
    disrespects_data[student] = disrespectful_students

print(disrespects_data)

with open("./Answering Questions/disrespects.json", "w") as file:
    json.dump(disrespects_data, file)

## Save graph using spectral layout
pos = nx.spectral_layout(G)
nt = Network()
nt.from_nx(G)

# Set node positions manually in Pyvis
for node, coords in pos.items():
    for node_dict in nt.nodes:
        if node_dict['id'] == node:
            node_dict['x'] = coords[0]
            node_dict['y'] = coords[1]
            break

# Save the graph with Spectral-like layout
nt.save_graph("./Layouts/Disrespects.html")

{'Derek Barnes': ['Derek Barnes', 'Miles Watson', 'Patrick Olsen'], 'Miles Watson': ['Miles Watson', 'Derek Barnes', 'Spencer Woodward'], 'Patrick Olsen': ['Patrick Olsen', 'Derek Barnes', 'Elias Cooper'], 'Spencer Woodward': ['Spencer Woodward', 'Miles Watson', 'Carter Baker'], 'Elias Cooper': ['Elias Cooper', 'Nikolai Weeks', 'Patrick Olsen'], 'Nikolai Weeks': ['Nikolai Weeks', 'Elias Cooper', 'Braxton Warren'], 'Jasper Luna': ['Jasper Luna', 'Victor Hodge', 'Mateo Ramirez'], 'Victor Hodge': ['Victor Hodge', 'Jasper Luna', 'Caden Novak'], 'Mateo Ramirez': ['Mateo Ramirez', 'Jasper Luna', 'Lorenzo Woods'], 'Lorenzo Woods': ['Lorenzo Woods', 'Mateo Ramirez', 'Jasper Luna'], 'Eli Collins': ['Eli Collins', 'Miles Watson', 'Nathan Allen'], 'James Anderson': ['James Anderson', 'Cole Stewart', 'Joseph Thompson'], 'Cole Stewart': ['Cole Stewart', 'James Anderson', 'Joseph Thompson'], 'King Ingram': ['King Ingram', 'Lawrence Cervantes', 'Victor Hodge'], 'Lawrence Cervantes': ['Lawrence Cervan

## Most Popular and influential Students

In [18]:
influentials = pd.read_excel(file_path, sheet_name="net_1_Influential")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
influentials.replace(IDtoName, inplace=True)

G = nx.Graph()

for i in range(len(influentials)):
    G.add_edge(influentials['Source'][i], influentials['Target'][i])


## Based on the graph, let us answer Who are the most influential students?
## This can be done by looking at the degree centrality of each node in the graph
degree_centrality = nx.degree_centrality(G)
influential_students = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:3]
influential_students = [student for student, degree in influential_students]
print(influential_students)





# with open("./Answering Questions/disrespects.json", "w") as file:
#     json.dump(advices_data, file)

# ## Save graph using spectral layout
# pos = nx.spectral_layout(G)
# nt = Network(notebook=True, select_menu=True)
# nt.from_nx(G)

# # Set node positions manually in Pyvis
# for node, coords in pos.items():
#     for node_dict in nt.nodes:
#         if node_dict['id'] == node:
#             node_dict['x'] = coords[0]
#             node_dict['y'] = coords[1]
#             break

# # Save the graph with Spectral-like layout
# nt.save_graph("./Layouts/Disrespects.html")

['Rory Briggs', 'Beckett Powell', 'Patrick Olsen']


## Students to spend more time with

In [19]:
student_times = pd.read_excel(file_path, sheet_name="net_3_MoreTime")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
student_times.replace(IDtoName, inplace=True)

G = nx.Graph()

for i in range(len(student_times)):
    G.add_edge(student_times['Source'][i], student_times['Target'][i])

times_data = dict()

for student in G.nodes:
    # Get the shortest paths from this student to all other students
    paths = nx.single_source_shortest_path(G, student)
    
    # The closest friends are those at a distance of 2 (i.e., one intermediate node)
    preffered_students = sorted([(friend, len(path)) for friend, path in paths.items()], key=lambda x: x[1])[:3]
    preffered_students = [advisor for advisor, distance in preffered_students]
    times_data[student] = preffered_students

print(times_data)

with open("./Answering Questions/spend_times.json", "w") as file:
    json.dump(times_data, file)

## Save graph using spectral layout
pos = nx.spectral_layout(G)
nt = Network()
nt.from_nx(G)

# Set node positions manually in Pyvis
for node, coords in pos.items():
    for node_dict in nt.nodes:
        if node_dict['id'] == node:
            node_dict['x'] = coords[0]
            node_dict['y'] = coords[1]
            break

# Save the graph with Spectral-like layout
nt.save_graph("./Layouts/Spend_Times.html")

{'Phillip Lozano': ['Phillip Lozano', 'Anders Crosby', 'Austin Foster'], 'Anders Crosby': ['Anders Crosby', 'Phillip Lozano', 'Austin Foster'], 'Connor Young': ['Connor Young', 'Jeremiah Myers', 'Julian King'], 'Jeremiah Myers': ['Jeremiah Myers', 'Connor Young', 'Ignatius Gaines'], 'Julian King': ['Julian King', 'Connor Young', 'Jonah Russell'], 'Kieran Pruitt': ['Kieran Pruitt', 'Connor Young', 'Samuel Harris'], 'Isaac Nelson': ['Isaac Nelson', 'Connor Young', 'Noah Johnson'], 'Ignatius Gaines': ['Ignatius Gaines', 'Connor Young', 'Jack Lopez'], 'Derek Barnes': ['Derek Barnes', 'Michael Jackson', 'Bryce Montgomery'], 'Michael Jackson': ['Michael Jackson', 'Derek Barnes', 'Jude Graham'], 'Elias Cooper': ['Elias Cooper', 'Andrew Martinez', 'Killian Kent'], 'Andrew Martinez': ['Andrew Martinez', 'Elias Cooper', 'Emmanuel Ford'], 'Killian Kent': ['Killian Kent', 'Elias Cooper', 'Damian Harrison'], 'Axel Griffin': ['Axel Griffin', 'Jordan Reed', 'Zayden Coleman'], 'Jordan Reed': ['Jordan 

## Students providing Meaningful Feedback

In [20]:
feedbacks = pd.read_excel(file_path, sheet_name="net_2_Feedback")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
feedbacks.replace(IDtoName, inplace=True)

G = nx.Graph()

for i in range(len(feedbacks)):
    G.add_edge(feedbacks['Source'][i], feedbacks['Target'][i])

feedbacks_data = dict()

for student in G.nodes:
    # Get the shortest paths from this student to all other students
    paths = nx.single_source_shortest_path(G, student)
    
    # The closest friends are those at a distance of 2 (i.e., one intermediate node)
    preffered_students = sorted([(friend, len(path)) for friend, path in paths.items()], key=lambda x: x[1])[:3]
    preffered_students = [advisor for advisor, distance in preffered_students]
    feedbacks_data[student] = preffered_students

print(feedbacks_data)

with open("./Answering Questions/feedbacks.json", "w") as file:
    json.dump(feedbacks_data, file)

## Save graph using spectral layout
pos = nx.spectral_layout(G)
nt = Network()
nt.from_nx(G)

# Set node positions manually in Pyvis
for node, coords in pos.items():
    for node_dict in nt.nodes:
        if node_dict['id'] == node:
            node_dict['x'] = coords[0]
            node_dict['y'] = coords[1]
            break

# Save the graph with Spectral-like layout
nt.save_graph("./Layouts/feedbacks.html")

{'Connor Young': ['Connor Young', 'Sebastian Gonzalez', 'Ignatius Gaines'], 'Sebastian Gonzalez': ['Sebastian Gonzalez', 'Connor Young', 'Ignatius Gaines'], 'Ignatius Gaines': ['Ignatius Gaines', 'Connor Young', 'Matthew Martin'], 'Derek Barnes': ['Derek Barnes', 'Zayden Coleman', 'Bryce Montgomery'], 'Zayden Coleman': ['Zayden Coleman', 'Derek Barnes', 'Caden Novak'], 'Kingston Hamilton': ['Kingston Hamilton', 'Flynn Stuart', 'Dominic Rogers'], 'Spencer Woodward': ['Spencer Woodward', 'Jason Bailey', 'Patrick Olsen'], 'Jason Bailey': ['Jason Bailey', 'Spencer Woodward', 'Patrick Olsen'], 'Elias Cooper': ['Elias Cooper', 'Andrew Martinez'], 'Andrew Martinez': ['Andrew Martinez', 'Elias Cooper'], 'Cameron Cox': ['Cameron Cox', 'Jonathan Evans', 'Raphael Barron'], 'Jonathan Evans': ['Jonathan Evans', 'Cameron Cox', 'Raphael Barron'], 'Rhett Barton': ['Rhett Barton', 'Levi Mitchell', 'Beckett Powell'], 'Levi Mitchell': ['Levi Mitchell', 'Rhett Barton', 'Zane Pena'], 'Beckett Powell': ['Be

## Participating in School Activities

In [21]:
activities = pd.read_excel(file_path, sheet_name="net_affiliation_0_SchoolActivit")
participants = pd.read_excel(file_path, sheet_name="participants")
KeepColumns = ['Participant-ID', 'First-Name', 'Last-Name']
participants = participants[KeepColumns]
participants['Name'] = participants['First-Name'] + ' ' + participants['Last-Name']
## ID to Name Mapping for participants
IDtoName = participants.set_index('Participant-ID')['Name'].to_dict()
activities.replace(IDtoName, inplace=True)

affiliations = pd.read_excel(file_path, sheet_name="affiliations")
KeepColumns = ['ID', 'Title']
affiliations = affiliations[KeepColumns]
## ID to Name Mapping for affiliations
IDtoName = affiliations.set_index('ID')['Title'].to_dict()
activities.replace(IDtoName, inplace=True)

activities_data = activities.groupby('Source')['Target'].apply(list).to_dict()

print(activities_data)
with open("./Answering Questions/Affiliations.json", "w") as file:
    json.dump(feedbacks_data, file)


## Build a graph from the data. Save graph based on target indegree(higher indegree = bigger brighter node)
G = nx.DiGraph() 

for i in range(len(activities)):
    G.add_edge(activities['Source'][i], activities['Target'][i])

nt = Network()
nt.from_nx(G)

nt.save_graph("./Layouts/Affiliations.html")

{'Aaron Edwards': ['Fitness/Health Club'], 'Adam Rivera': ['Community Service Club', 'Math Club'], 'Adrian Cook': ['Astronomy Club', 'Diversity and Inclusion Club'], 'Aiden Brown': ['National Honor Society'], 'Alexander Taylor': ['Photography Club'], 'Anders Crosby': ['Film Club', 'Astronomy Club', 'Diversity and Inclusion Club'], 'Andrew Martinez': ['National Honor Society', 'Science Club', 'Film Club'], 'Anthony Hill': ['Photography Club', 'Debate Team'], 'Atticus Osborne': ['Photography Club', 'National Honor Society'], 'Austin Foster': ['Gardening Club', 'Photography Club'], 'Axel Bennett': ['Photography Club'], 'Axel Griffin': ['Photography Club'], 'Barrett Gibson': ['National Honor Society'], 'Beckett Powell': ['Chess Team', 'Debate Team', 'National Honor Society', 'History Club', 'Technology Club'], 'Benjamin Miller': ['Technology Club', 'History Club'], 'Bennett Sullivan': ['Photography Club', 'Science Club', 'Gardening Club'], 'Bentley Price': ['Fitness/Health Club', 'Math Clu